In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-10-23 04:33:06.250762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-10-23 04:33:06.250810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
runs = 5

ds_name = 'sst2'
test_dataset = None
eval_mapping = '[]'
num_labels = 2
valid_split = 'validation'
test_split = 'test'

ranking = 0
db_path = "./genetic_deepset_search.db"
seed = 0
pool_size = 1000
search_size = 100
warmup_runs = 500
annealing_runs = 1000
wandb_project = 'sst_search_test'
wandb_entity = 'johntzwei'

model_card = "google/electra-small-discriminator"
pretraining = True
max_steps = 3000
eval_steps = 500
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
training_args = select.SubsetTrainingArguments(num_labels=num_labels,
                                               model_card=model_card,
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(db_path=db_path,
                                                seed=seed, 
                                                data_pool_size=pool_size,
                                                optimal_subset_size=search_size,
                                                warmup_runs=warmup_runs,
                                                annealing_runs=annealing_runs,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [5]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-5e912404f35e1845.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-a8efe87a750447a6.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-ab311af0e13f6190.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-abedee5dbca0c22b.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0e96b66df4e50237.arrow
Loading cached processed dataset at /home/johnny/.cache

In [6]:
processed_ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2210
    })
})

In [7]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds[valid_split], test_ds=processed_ds[test_split]
)

In [8]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(db_path)
df = df.sort_values('objective')
idx = json.loads(df.iloc[-ranking-1]['indexes'])

In [9]:
df

,indexes,objective
0,"[107, 748, 732, 767, 692, 363, 737, 273, 848, ...",0.000000
206,"[505, 96, 984, 211, 809, 766, 856, 313, 983, 9...",0.709717
473,"[671, 213, 370, 996, 160, 715, 471, 754, 631, ...",0.712192
27,"[612, 238, 350, 196, 913, 230, 613, 135, 682, ...",0.713052
283,"[614, 894, 599, 855, 36, 671, 318, 583, 554, 1...",0.714268
...,...,...
5785,"[763, 453, 894, 976, 195, 554, 789, 68, 34, 53...",0.772830
5493,"[329, 859, 583, 745, 620, 709, 237, 296, 649, ...",0.772853
5934,"[492, 389, 453, 72, 649, 962, 857, 456, 858, 3...",0.772883
5671,"[117, 509, 338, 879, 152, 759, 497, 892, 352, ...",0.772900


In [10]:
len(idx)

100

In [11]:
pool_dataset = processed_ds['train'].shuffle(seed=seed).select(range(1000, 2000))

In [17]:
select_quality = []
for i in range(runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    wandb.log({"nth" : len(df) - ranking, "db" : db_path})
    
    train_dataset = pool_dataset.select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    select_quality.append(quality)
    
    wandb.log({"quality" : quality})

loss,████▇▇▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
quality,▁
ranking,▁
sst2_test:accuracy,▁
sst:val_acc,▁▇██
db,./genetic_deepset_se...
loss,0.00393
quality,0.73665
ranking,0
sst2_test:accuracy,0.73665
sst:val_acc,0.72025


wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-10-23 04:51:12.539265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-10-23 04:51:12.539312: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

...

  0%|          | 0/3000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
select_quality

[0.7366515837104073,
 0.7153846153846154,
 0.7199095022624434,
 0.7244343891402715,
 0.7366515837104073]

In [16]:
np.mean(select_quality), np.std(select_quality)

(0.7266063348416291, 0.008686840069514198)